<center>
<h1><b>A Review on LSTM FCN for Time Series Classification</b></h1>
<h2>Authors: Alex Wei, Junyu Hu, Xudong Chen
</center>

<br>

<span><h2>__README__</h2></span><br>
1. Recommended to run on GCP (preferably NVIDIA A100 at minimum if you want to train all models, otherwise any device should suffice for pure evaluation). If you intend to run codes in Google Colab, please first click Runtime tab and change the runtime type to GPU; then make appropriate changes to mount your drive and access other python scripts. Anaconda (with tf <= 2.10) / PyCharm / CUDA 11.x is recommended if choosing local env.

2. For the dataset, please download it from our [Google Drive](https://drive.google.com/drive/u/1/folders/1YlGx9RX7Q5g9SMh2_VTB02YKwCNXuKqZ) (for this project, we used the newest 2018 version) to the same directory as this notebook. You may also refer to [UCR Time Series Classification Archive](https://www.cs.ucr.edu/~eamonn/time_series_data_2018/). However note that it is encrypted (password == `someone`) and slightly different from our version (see 3 why).

3. Please be advised that we extract relevant dataset info from DataSummary.csv, which should be automatically cloned into the current directory. However, it has been modified based on the one that you directly download from UCR Archive, since as they mentioned, some datasets have missing values and varying time series length (see details in `Missing_value...` folder once you decompress). In order to give reproducible results, we here manually updated them. SO DEAR TA PLEASE USE OUR VERSION OF THIS CSV AND DATASET.

# Change Log
- 11/28 @xc2763: Created data extraction script
- 11/29 @yw4467: Integrated data extraction into main.py; initiated the project organization on GitHub/Colab/Google Doc
- 11/30 @yw4467: Full optimization on the project structure; created const.py to automate the info extraction
- 11/30 @yw4467: Simplified dataset info by using a dictionary in const.py; added comments to improve readability
- 12/01 @yw4467: Updated DataSummary.csv and UCR Archive dataset to ensure reproducibility
- 12/02 @yw4467: Migrated visualization codes from author's repo; improved robustness of data extraction
- 12/03 @yw4467: Implemented customized Attention LSTM (`./utils/attention.py`)
- 12/03 @xc2763: Core bug fix for attention mechanism
- 12/05 @yw4467: Fully optimized Attention LSTM
- 12/05 @yw4467: Compatibility & readability improvement for TA
- 12/12 @xc2763: Implemented data loading function in (`./utils/generics.py`)
- 12/13 @xc2763: Loss function bug fix for model evaluation

__Planned Milestones:__
- 12/10 @yw4467: Complete first trained models
- 12/12 @xc2763: Implement generic functions
- 12/12 @jh4930: Implement utility functions
- 12/14 @xc2763: Implement plot dataset function in (`./utils/generics.py`)

In [ ]:
# ! pip install -r requirements.txt

In [ ]:
import os

import tensorflow as tf
from tensorflow.keras import backend, Model, layers as l

# snippet modified from Assignment 3
dev = tf.config.list_physical_devices('GPU')
if len(dev) > 0:
    gpu = tf.config.experimental.get_device_details(dev[0])
    print('Active GPU(0):', gpu['device_name'])
    tf.config.experimental.set_memory_growth(dev[0], enable=True)
else:
    print('Running only on CPU')

# Initial data Extraction
<span style="color: cyan">@ Xudong Chen & Alex Wei</span> <br>
Run this after you put the compressed dataset into current directory. If you would like to replicate results as the original paper by using the 2015 version, modify `PATH` below.

In [ ]:
import glob
import pandas as pd
from joblib import Parallel, delayed
import zipfile

PATH = 'UCRArchive_2018.zip'    # expanded, compared to 2015 version used in the original paper
csv = 'data'                    # directory of parsed CSVs; modify as you see fit

# unzip if not yet (the zip should have been auto downloaded to current dir)
if not os.path.exists(os.path.splitext(PATH)[0]):
    with zipfile.ZipFile(PATH, 'r') as zip_ref:
        zip_ref.extractall('.')

if not os.path.exists(csv): # create output directory if doesn't exist
    os.makedirs(csv)

def extract_raw(file):
    """Convert each raw data file (.tsv) into .csv"""
    out = os.path.join(csv, os.path.splitext(os.path.basename(file))[0] + '.csv')   # strip .tsv ext
    df = pd.read_table(file, header=None, encoding='latin-1')   # load values in the dataset
    df.fillna(0.0, inplace=True)    # fill empty time steps
    df.to_csv(out, index=False, header=None, encoding='latin-1')

files = glob.glob(os.path.join(os.path.splitext(PATH)[0], '**', '*.tsv'), recursive=True)   # find all tsv
if files:
    # check if all corresponding CSVs already exist
    existing = set(glob.glob(os.path.join(csv, '*.csv')))
    expected = {os.path.join(csv, os.path.splitext(os.path.basename(f))[0] + '.csv') for f in files}

    if existing == expected:
        print(f'All {len(files)} tsv have already been parsed. Skipping conversion.')
    else:
        print(f'{len(files)} tsv found. Processing...')
        # TODO : progress bar
        with Parallel(n_jobs=-1) as engine: engine([delayed(extract_raw)(file) for file in files])
        print(f'All processed / saved to {csv}')
else:
    print('Error locating tsv files in the specified directory.')

# Main - Training and Evaluation
<span style="color: cyan">@ Alex Wei</span>

In [ ]:
from utils.const import META
from utils.keras_utils import train, eval
from utils.attention import ALSTM
from tensorflow.keras.initializers import Orthogonal

init = Orthogonal(seed=42)  # expecting slight better performance than he_uniform

# 3 layers as described in the paper
def gen(len_ts, n_class, n_cell=8, use_att=False):
    """ Generate the model for training with given method
    Args:
        len_ts: time series length;
        n_cell: number of cells in the LSTM layer;
        n_class: number of classes in the dataset;
        use_att: enable attention mechanism?
    """
    m = l.Input(shape=(1, len_ts))

    # tf 2.x auto uses cuDNN LSTM if available
    x = ALSTM(n_cell)(m) if use_att else l.LSTM(n_cell, recurrent_activation='sigmoid')(m)
    x = l.Dropout(0.8)(x)

    y = l.Permute((2, 1))(m) # dimension shuffle
    y = l.Conv1D(128, 8, padding='same', kernel_initializer=init)(y)
    y = l.BatchNormalization()(y)
    y = l.Activation('relu')(y)
    y = l.Conv1D(256, 5, padding='same', kernel_initializer=init)(y)
    y = l.BatchNormalization()(y)
    y = l.Activation('relu')(y)
    y = l.Conv1D(128, 3, padding='same', kernel_initializer=init)(y)
    y = l.BatchNormalization()(y)
    y = l.Activation('relu')(y)
    y = l.GlobalAveragePooling1D()(y)

    x = l.concatenate([x, y])
    out = l.Dense(n_class, activation='softmax')(x)

    return Model(m, out)


if __name__ == '__main__':
    models = [('alstmfcn', True), ('lstmfcn', False)]
    cells = [8, 64, 128]  # number of cells

    for name, att in models:
        for cell in cells:
            success = []

            log = f'{name}_{cell}cell_summary.csv'  # log all training results
            if not os.path.exists(log):
                with open(log, 'w') as file:
                    file.write('ID, Dataset, Weight Path, Test Accuracy\n')

            for dataset in META:
                backend.clear_session()  # release VRAM
                file = open(log, 'a+')
                entry, ID = dataset['Name'], dataset['ID'] - 1
                dir_weight = f'{name}_{cell}cell/{entry}'
                os.makedirs(f'weights/{os.path.dirname(dir_weight)}', exist_ok=True)

                model = gen(dataset['Length'], dataset['Class'], cell, use_att=att)
                print(f'{">" * 16} Training Dataset: {entry} {ID + 1}/{len(META)} {"<" * 16}')
                # NOTE FOR TA: comment out line below for mere evaluation
                train(model, ID, dir_weight, epochs=2400, batch_size=128, norm_ts=True)
                acc = eval(model, ID, dir_weight, batch_size=128, norm_ts=True)
                result = f'{ID + 1}, {entry}, {dir_weight}, {acc:.6f}\n'
                file.write(result)
                file.flush()
                success.append(result)
                file.close()

            print(f'\n{">" * 16} TRAINING COMPLETE {"<" * 16}')
            for line in success:
                print(line)

# Visualizations
<span style="color: cyan">@ all</span>
This section directly uses part of the source code from the author [1].

In [ ]:
from utils.keras_utils import visualize_filters, visualize_cam, visualize_context_vector

# TODO: implement our own visualization functions, e.g. comparing our acc with original
# COMMON PARAMETERS
DATASET_ID = 0
num_cells = 8

# NEW 43 DATASET PARAMETERS
model_name = 'alstmfcn'

# visualization params
CLASS_ID = 0
CONV_ID = 0
FILTER_ID = 0
LIMIT = 1
VISUALIZE_SEQUENCE = True
VISUALIZE_CLASSWISE = False

# script setup
sequence_length = META[DATASET_ID]['Length']
nb_classes = META[DATASET_ID]['Class']
model = gen(sequence_length, nb_classes, num_cells, use_att=False)

entry = META[DATASET_ID]['Name']
name_ = f'{model_name}_{num_cells}cell/{entry}'

visualize_cam(model, DATASET_ID, name_, class_id=CLASS_ID, seed=0,
              normalize_timeseries=True)
visualize_context_vector(model, DATASET_ID, name_, limit=LIMIT, visualize_sequence=VISUALIZE_SEQUENCE,
                             visualize_classwise=VISUALIZE_CLASSWISE, normalize_timeseries=True)
visualize_filters(model, DATASET_ID, name_, conv_id=CONV_ID, filter_id=FILTER_ID, seed=0,
                      normalize_timeseries=True)

# References
[1] Karim, Fazle, et al. "[LSTM fully convolutional networks for time series classification.](https://github.com/titu1994/LSTM-FCN)" IEEE Access 7 (2019): 10127-10137. <br>
[2] Wang, Zongwei, et al. "[Time series classification from scratch with deep neural networks: A strong baseline.](https://arxiv.org/abs/1611.06455)" 2016.